# CNC專案 - CyberInsight幹嘛我們就幹嘛
- 富士康汇报5.11.pdf

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, datetime, pickle, warnings
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#中文字型
fp = matplotlib.font_manager.FontProperties(fname = 'font/NotoSansCJKtc-Regular.otf')
matplotlib.font_manager.fontManager.ttffiles.append(fp.get_file())
font_entry = matplotlib.font_manager.FontEntry(fp.get_file(), name=fp.get_name(),
                                               style=fp.get_style(), variant=fp.get_variant(),
                                              weight=fp.get_weight(), stretch=fp.get_stretch(), size=fp.get_size())

matplotlib.font_manager.fontManager.ttflist.append(font_entry)
plt.rcParams['font.family'] = fp.get_name()

from scipy.fftpack import fft, ifft
import pylab
import seaborn as sns
from nptdms import TdmsFile
import charts
from scipy.interpolate import spline
from scipy import signal
from scipy.signal import freqz, butter, lfilter
import datetime
from scipy.stats import skew, kurtosis
from sklearn import decomposition
import charts
pd.set_option('display.width', 2000)

warnings.filterwarnings("ignore")



%matplotlib inline

Server running in the folder /Users/jayhsu/work/github/CDProj at 127.0.0.1:56410


** read data functions **

In [2]:
def _readTDMSasDF(path):
    tdms_file = TdmsFile(path)
    tdms_df = tdms_file.as_dataframe()
    return tdms_df
def readTDMSasDF(path):
    tdms_file = TdmsFile(path)
    #tdms_df = tdms_file.as_dataframe()
    data_dict = {}
    time_dict= {}
    for p in tdms_file.objects.keys():
        path_list = list(filter(None, p.split('/')))
        path_list = list(map(lambda s: s.replace("'", ""), path_list ))
        if not path_list:
            continue
        if len(path_list)==1:
            continue
        if len(path_list)==2:
            obj=tdms_file.object(path_list[0],path_list[1])
            data = obj.data
            time = obj.time_track()
            data_dict['_'.join(path_list)] = data
            time_dict['_'.join(path_list)] = time
    return pd.DataFrame(data_dict, index = list(time_dict.values())[0])


## 主軸異常建模 - FCFT振动信号
- Data: LH_FCFT/FCFT-20180507

### 數據切割 - Stationary Data

In [3]:
def getPLCData(filepath, datestr):
    plc_df = pd.read_csv(filepath)
    plc_df['Time(ms)'] = plc_df['Time(ms)'].map(lambda s: s.strip())
    print(plc_df.shape)
    plc_df['Time(ms)']=plc_df['Time(ms)'].map(lambda x: datestr+' '+x)
    plc_df['Time(ms)']=pd.to_datetime(plc_df['Time(ms)'], format="%Y%m%d %H:%M:%S:%f") #13:43:42:550
    plc_df = plc_df.set_index('Time(ms)')
    plc_df = plc_df.apply(pd.to_numeric, errors='ignore')
    plc_df = plc_df.fillna(0)
    return plc_df

#plc_df_0507=getPLCData('data/LH_FCFT/FCFT-20180507/PLC/20180507_081232.csv', '20180507')
#plc_df_0508=getPLCData('data/LH_FCFT/FCFT-20180508/PLC/20180508_081044.csv', '20180508')
           


def getSensorData(folder_path):
    tdmsfilelist = []
    for f in os.listdir(folder_path):
        if not f.endswith('tdms'):
            continue
        else:
            tdmsfilelist.append(f)

    tdmsfilelist = sorted(tdmsfilelist)
    if True:
        df_list=[]
        for f in tdmsfilelist:
            try:
                path = '{}/{}'.format(folder_path, f)
                df_ = readTDMSasDF(path)
                tstr = f[4:-5]
                dtobj = datetime.datetime.strptime(tstr, '%y%m%d%H%M%S')
                df_['time'] = list(map(lambda x: x[0] + datetime.timedelta(0,x[1]), list(zip([dtobj]*len(df_.index), df_.index))))
                df_list.append(df_)
            except:
                #print('except',folder,path, os.path.getsize(path)/1024/1024)
                pass
    tdms_df = pd.concat(df_list, ignore_index=True)
    return tdms_df

def getStationaryData(plc_df, sensordata, speed):
    dur = plc_df[plc_df['spindle command S(rpm)']==speed].index
    endtime = max(dur)
    dur = plc_df[plc_df['spindle real S(rpm)']>=speed].index
    starttime = min(dur)
    tdata = sensordata[(sensordata['time']>=starttime) & (sensordata['time']<=endtime)]
    return tdata

plc_df_0507=getPLCData('data/LH_FCFT/FCFT-20180507/PLC/20180507_081232.csv', '20180507')
tdms_df_0507=getSensorData('data/LH_FCFT/FCFT-20180507/')     
stationary_11k_df = getStationaryData(plc_df_0507, tdms_df_0507, 11000)
stationary_11k_df['speed'] = 11000
del tdms_df_0507

(53825, 14)


In [4]:
stationary_11k_df = stationary_11k_df.reset_index()
stationary_11k_df = stationary_11k_df.set_index('time')
stationary_11k_df = stationary_11k_df[['Spindle_S01','Spindle_S02','Spindle_S03']]
#stationary_11k_df['timestamp'] = stationary_11k_df['time'].map(lambda x: x.timestamp())
#stationary_11k_df['timestamp_new'] =  stationary_11k_df['timestamp'] - stationary_11k_df['timestamp'].iloc[0]
#stationary_11k_df = stationary_11k_df.set_index('timestamp_new')
#stationary_11k_df = stationary_11k_df.reset_index()

In [5]:
stationary_11k_df.head()

,Spindle_S01,Spindle_S02,Spindle_S03
time,,,
2018-05-07 08:18:02.417031,-0.400293,0.469737,0.649388
2018-05-07 08:18:02.417070,-0.087023,-0.705409,-1.103354
2018-05-07 08:18:02.417109,1.133183,0.113063,1.439153
2018-05-07 08:18:02.417148,-1.234783,-0.072352,-1.335240
2018-05-07 08:18:02.417187,0.253566,0.376511,0.928236


In [11]:
options = {
    'title': {'text': 'raw data'}
}

charts.plot(stationary_11k_df[:100], options=options, show='inline', stock=True)



** feature extraction **

In [14]:
def fea_rms(d):
    return np.sqrt(np.square(d).mean())
    
def fea_peaktopeak(d):
    return (max(d) - min(d))

def fea_skewness(d):
    return skew(d)

def fea_kurtosis(d):
    return kurtosis(d)

def fea_x1tox10(d):
    pass

def fea_rpmorder(data,**kwargs):
    rpm, sr, rpmorder=kwargs['rpm'], kwargs['sr'], kwargs['rpmorder']
    #rpm=11000, sr=25600, rpmorder=1
    rmpHz = rpm/60
    fft_size = len(data)
    window = np.hanning(fft_size) 
    N = fft_size
    T = 1.0 /sr
    x = np.linspace(0.0, N*T, N)
    y = data
    y=y*window
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    xf_order = xf//rmpHz
    yf = 2.0/N * np.abs(yf[0:N//2])
    return yf[np.where(xf_order==rpmorder)[0].min()]
    


In [15]:
stationary_11k_rms = stationary_11k_df.resample('5s').apply(fea_rms)
stationary_11k_rms.rename(columns={'Spindle_S01': 'S01_RMS', 'Spindle_S02': 'S02_RMS','Spindle_S03':'S03_RMS'}, inplace=True)

stationary_11k_p2p = stationary_11k_df.resample('5s').apply(fea_peaktopeak)
stationary_11k_p2p.rename(columns={'Spindle_S01': 'S01_P2P', 'Spindle_S02': 'S02_P2P','Spindle_S03':'S03_P2P'}, inplace=True)

stationary_11k_skewness = stationary_11k_df.resample('5s').apply(fea_skewness)
stationary_11k_skewness.rename(columns={'Spindle_S01': 'S01_skewness', 'Spindle_S02': 'S02_skewness','Spindle_S03':'S03_skewness'}, inplace=True)

stationary_11k_kurtosis = stationary_11k_df.resample('5s').apply(fea_kurtosis)
stationary_11k_kurtosis.rename(columns={'Spindle_S01': 'S01_kurtosis', 'Spindle_S02': 'S02_kurtosis','Spindle_S03':'S03_kurtosis'}, inplace=True)

stationary_11k_1x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=1)
stationary_11k_1x.rename(columns={'Spindle_S01': 'S01_1x', 'Spindle_S02': 'S02_1x','Spindle_S03':'S03_1x'}, inplace=True)

stationary_11k_2x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=2)
stationary_11k_2x.rename(columns={'Spindle_S01': 'S01_2x', 'Spindle_S02': 'S02_2x','Spindle_S03':'S03_2x'}, inplace=True)

stationary_11k_3x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=3)
stationary_11k_3x.rename(columns={'Spindle_S01': 'S01_3x', 'Spindle_S02': 'S02_3x','Spindle_S03':'S03_3x'}, inplace=True)

stationary_11k_4x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=4)
stationary_11k_4x.rename(columns={'Spindle_S01': 'S01_4x', 'Spindle_S02': 'S02_4x','Spindle_S03':'S03_4x'}, inplace=True)

stationary_11k_5x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=5)
stationary_11k_5x.rename(columns={'Spindle_S01': 'S01_5x', 'Spindle_S02': 'S02_5x','Spindle_S03':'S03_5x'}, inplace=True)

stationary_11k_6x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=6)
stationary_11k_6x.rename(columns={'Spindle_S01': 'S01_6x', 'Spindle_S02': 'S02_6x','Spindle_S03':'S03_6x'}, inplace=True)

stationary_11k_7x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=7)
stationary_11k_7x.rename(columns={'Spindle_S01': 'S01_7x', 'Spindle_S02': 'S02_7x','Spindle_S03':'S03_7x'}, inplace=True)

stationary_11k_8x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=9)
stationary_11k_8x.rename(columns={'Spindle_S01': 'S01_8x', 'Spindle_S02': 'S02_8x','Spindle_S03':'S03_8x'}, inplace=True)

stationary_11k_9x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=9)
stationary_11k_9x.rename(columns={'Spindle_S01': 'S01_9x', 'Spindle_S02': 'S02_9x','Spindle_S03':'S03_9x'}, inplace=True)

stationary_11k_10x = stationary_11k_df.resample('5s').apply(fea_rpmorder, rpm=11000, sr=25600, rpmorder=10)
stationary_11k_10x.rename(columns={'Spindle_S01': 'S01_10x', 'Spindle_S02': 'S02_10x','Spindle_S03':'S03_10x'}, inplace=True)

stationary_11k_features = pd.concat([stationary_11k_rms, stationary_11k_p2p, stationary_11k_skewness, stationary_11k_kurtosis, stationary_11k_1x, stationary_11k_2x, stationary_11k_3x, stationary_11k_4x, stationary_11k_5x, stationary_11k_6x, stationary_11k_7x, stationary_11k_8x, stationary_11k_9x, stationary_11k_10x], axis=1)


In [17]:
stationary_11k_features.head()

,S01_RMS,S02_RMS,S03_RMS,S01_P2P,S02_P2P,S03_P2P,S01_skewness,S02_skewness,S03_skewness,S01_kurtosis,...,S03_7x,S01_8x,S02_8x,S03_8x,S01_9x,S02_9x,S03_9x,S01_10x,S02_10x,S03_10x
time,,,,,,,,,,,,,,,,,,,,,
2018-05-07 08:18:00,0.533658,0.399788,0.765709,3.913578,3.230631,7.246757,0.008502,0.093217,0.016683,-0.130439,...,0.000060,0.000050,0.000028,0.000007,0.000050,0.000028,0.000007,0.000076,0.000085,0.000046
2018-05-07 08:18:05,0.680316,0.425144,0.705663,5.605548,3.209276,6.148564,0.000060,-0.011651,-0.036299,-0.011250,...,0.000262,0.000285,0.000336,0.000101,0.000285,0.000336,0.000101,0.000602,0.000493,0.000155
2018-05-07 08:18:10,0.644192,0.432804,0.732435,4.818991,3.628244,6.002614,0.018947,-0.012391,-0.023115,-0.328249,...,0.000157,0.000071,0.000115,0.000061,0.000071,0.000115,0.000061,0.000050,0.000025,0.000009
2018-05-07 08:18:15,0.607267,0.429811,0.680559,5.196231,3.468759,5.487794,-0.028044,0.044180,0.010025,0.127998,...,0.000109,0.000013,0.000035,0.000059,0.000013,0.000035,0.000059,0.000041,0.000109,0.000061
2018-05-07 08:18:20,0.749788,0.437994,0.716418,5.613537,3.480229,5.436684,-0.003858,-0.000150,-0.014307,-0.146801,...,0.000076,0.000020,0.000026,0.000032,0.000020,0.000026,0.000032,0.000080,0.000117,0.000067


In [16]:
options = {
    'title': {'text': 'All Features'}
}

charts.plot(stationary_11k_features, options=options, show='inline', stock=True)



In [ ]:
pca = decomposition.PCA(n_components=0.95)
X= stationary_11k_features.values
pca.fit(X)
print('n_components_',pca.n_components_)
print('explained_variance_',pca.explained_variance_)
print('explained_variance_ratio_',pca.explained_variance_ratio_)
#驗證一下T=XP
#mean=np.array([np.mean(X[:,i]) for i in range(4)])
#norm_X=X-mean
#np.dot(norm_X,np.transpose(pca.components_))
P = pca.components_.T


** T2 **

In [ ]:
eigvDiagInv = np.linalg.inv(np.diag(pca.explained_variance_))
T2_list = []
for xraw in X: 
    x=xraw-pca.mean_
    t2 = np.dot(np.dot(np.dot(np.dot(x, P),eigvDiagInv),P.T), x.T)
    T2_list.append(t2)
    
plt.plot(T2_list)
plt.title('T2')

** SPE **

In [ ]:
ppt= np.matmul(P,P.T)
ippt = np.diag([1]*len(ppt)) - ppt
spe_list=[]
for xraw in X: 
    x=xraw-pca.mean_
    q = np.dot(np.dot(x, ippt), x.T)
    spe_list.append(q)
plt.plot(spe_list)
plt.title('SPE')